# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, precision_score
from sklearn.metrics import f1_score, recall_score, precision_recall_curve, roc_curve
from sklearn.metrics import accuracy_score, roc_auc_score, ConfusionMatrixDisplay
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head().transpose()

,0,1,2,3,4
age,56,57,37,40,56
job,housemaid,services,services,admin.,services
marital,married,married,married,married,married
education,basic.4y,high.school,high.school,basic.6y,high.school
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
month,may,may,may,may,may
day_of_week,mon,mon,mon,mon,mon


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [5]:
#Create Dataframe with the relevant features
bank_df = df[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'y']].copy()
bank_df['y'].value_counts()

y
no     36548
yes     4640
Name: count, dtype: int64

In [6]:
# convert categorical columns into numerical data
bank_df = pd.get_dummies(bank_df, columns=['job', 'marital', 'education', 'contact'], drop_first=True)
bank_df['y'] = bank_df['y'].map({'yes': 1, 'no': 0})
bank_df['housing'] = bank_df['housing'].map({'yes': 1, 'no': 0})
bank_df['default'] = bank_df['default'].map({'yes': 1, 'no': 0})
bank_df['loan'] = bank_df['loan'].map({'yes': 1, 'no': 0})
bank_df.dropna(inplace=True)
bank_df.head()

,default,housing,loan,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,...,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,contact_telephone
0,0.0,0.0,0.0,0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,0.0,1.0,0.0,0,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
3,0.0,0.0,0.0,0,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
4,0.0,0.0,1.0,0,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
6,0.0,0.0,0.0,0,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [7]:
from sklearn.model_selection import train_test_split
X = bank_df.drop('y', axis=1)
y = bank_df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting splits
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (25462, 25)
Test data shape: (6366, 25)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [8]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

baseline = DummyClassifier(strategy='most_frequent' , constant=0).fit(X_train, y_train)
baseline_pred = baseline.predict(X_test)
baseline_accuracy = accuracy_score(baseline_pred, y_test)
print(f'Baseline accuracy score: {baseline_accuracy}')

# Baseline accuracy score with strategy most_frequent: 0.8704052780395853 
# Baseline accuracy score with strategy uniform: 0.4949732956330506 
# Baseline accuracy score with strategy stratified: 0.776468740182218
# Baseline accuracy score with strategy constant: 0.8704052780395853

Baseline accuracy score: 0.8704052780395853


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [9]:
from sklearn.linear_model import LogisticRegression
import time

simple_lgr = LogisticRegression()

start = time.time()
simple_lgr.fit(X_train, y_train)
stop = time.time()
fit_time = str(round(stop-start,2))+'s'
print("fit time", fit_time)

fit time 0.27s


### Problem 9: Score the Model

What is the accuracy of your model?

In [10]:
y_pred = simple_lgr.predict(X_test)

lgr_accuracy = accuracy_score(y_pred, y_test)
print(f'Logistic accuracy score: {lgr_accuracy}')

Logistic accuracy score: 0.8704052780395853


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [11]:
models = {
    'LogisticRegression': LogisticRegression(),
    'KNC': KNC(),
    'SVC': SVC(),
    'DTC': DTC()
}

# Print the header
print(f"{'Model':<20}{'Train Time':<15}{'Train Accuracy':<18}{'Test Accuracy'}")

# Iterate over models and calculate the details
for model_name, model in models.items():
    # Start the timer for training
    start_time = time.time()
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # Calculate the training time
    train_time = time.time() - start_time

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Print the model details in the desired format
    print(f"{model_name:<20}{train_time:<15.4f}{train_accuracy:<18.4f}{test_accuracy}")


Model               Train Time     Train Accuracy    Test Accuracy
LogisticRegression  0.2225         0.8713            0.8704052780395853
KNC                 0.0130         0.8597            0.8517122211749921
SVC                 18.1604        0.8713            0.8704052780395853
DTC                 0.0491         0.8753            0.8660069117185045


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [12]:
bank_df_full = df.copy()
bank_df_full = pd.get_dummies(bank_df_full, columns=['job', 'marital', 'education', 'contact', 'month', 'day_of_week', 'poutcome'], drop_first=True)
bank_df_full['y'] = bank_df_full['y'].map({'yes': 1, 'no': 0})
bank_df_full['housing'] = bank_df_full['housing'].map({'yes': 1, 'no': 0})
bank_df_full['default'] = bank_df_full['default'].map({'yes': 1, 'no': 0})
bank_df_full['loan'] = bank_df_full['loan'].map({'yes': 1, 'no': 0})
bank_df_full.dropna(inplace=True)
bank_df_full.head()

#train test split
X = bank_df_full.drop('y', axis=1)
y = bank_df_full['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
models = {
    'LogisticRegression': LogisticRegression(),
    'KNC': KNC(),
    'SVC': SVC(),
    'DTC': DTC()
}

# Print the header
print(f"{'Model':<20}{'Train Time':<15}{'Train Accuracy':<18}{'Test Accuracy'}")

# Iterate over models and calculate the details
for model_name, model in models.items():
    # Start the timer for training
    start_time = time.time()
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # Calculate the training time
    train_time = time.time() - start_time

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Print the model details in the desired format
    print(f"{model_name:<20}{train_time:<15.4f}{train_accuracy:<18.4f}{test_accuracy}")

Model               Train Time     Train Accuracy    Test Accuracy
LogisticRegression  0.3592         0.8965            0.8986804901036758
KNC                 0.0232         0.9217            0.8942821237825951
SVC                 18.3038        0.8841            0.8845428840716305
DTC                 0.2259         1.0000            0.8741753063147973


In [ ]:
# GridSearchCV on LogisticRegression to verify the accuracy
log_rg = LogisticRegression()

param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 1000, 10000],
    'C': [0.1, 1, 10, 100, 1000]
}

grid_log = GridSearchCV(log_rg, param_grid, refit = True)
grid_log.fit(X_train, y_train)
print(grid_log.best_params_)
print(grid_log.best_estimator_)
grid_log_pred = grid_log.predict(X_test)
logistic_grid_accuracy = accuracy_score(grid_log_pred, y_test)
print(f'Grid accuracy: {logistic_grid_accuracy}')


In [ ]:
knn = KNC()
param_grid = {'n_neighbors': np.arange(1, 25), 
              'weights': ['uniform', 'distance'], 
              'metric': ['euclidean', 'manhattan'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
              }

grid_knn = GridSearchCV(knn, param_grid, refit = True)
grid_knn.fit(X_train, y_train)
print(grid_knn.best_params_)
print(grid_knn.best_estimator_)
print(grid_knn.best_params_)
grid_knn_pred = grid_knn.predict(X_test)
grid_knn_accuracy = accuracy_score(grid_knn_pred, y_test)
print(f'Grid accuracy: {grid_knn_accuracy}')


In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid_svc = GridSearchCV(SVC(), param_grid, refit = True) 

grid_svc.fit(X_train, y_train) 
print(grid_svc.best_params_)
best_params = {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
print(grid_svc.best_estimator_)
print(grid_svc.best_params_)
grid_svc_pred = grid_svc.predict(X_test)
grid_svc_accuracy = accuracy_score(grid_svc_pred, y_test)
print(f'Grid accuracy: {grid_svc_accuracy}')

In [ ]:
dtc = DTC()

param_grid = { 
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth' : [i for i in range(1, 11)]
}

grid_dtc = GridSearchCV(dtc, param_grid, refit = True)
grid_dtc.fit(X_train, y_train)
print(grid_dtc.best_params_)
print(grid_dtc.best_estimator_)
grid_dtc_pred = grid_dtc.predict(X_test)
grid_dtc_accuracy = accuracy_score(grid_dtc_pred, y_test)
print(f'Grid accuracy: {grid_dtc_accuracy}')

##### Questions
The testing of different models with limited features and and with more no of features are computed, and based on the results considering the Train Accuracy, with limited features, LogisticRegression performed better with accuracy of 0.8713 where as with full features, KNearestClassifiers performed best with high accuracy of 0.9217. I tried to compute the accuracy with GridSearchCV for the all the models, but it is taking time and results are not shown yet, hence not able to comment out on this.